In [17]:
import imaplib
import email
import mysql.connector
import time

In [18]:
username = 'rinkalranjan@outlook.com'
password = 'R@12rinkal'


In [20]:
# establish connection to Outlook mail server
imap_server = 'imap-mail.outlook.com'
mail = imaplib.IMAP4_SSL(imap_server)
mail.login(username, password)
mail.select("inbox")

('OK', [b'2'])

In [21]:
# search for all emails in inbox
result, data = mail.search(None, "ALL")

In [22]:
# connect to MySQL database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="rinkal",
  database="samsung"
)
mycursor = mydb.cursor()

In [23]:
# loop through emails and insert into database
for num in data[0].split():
    typ, data = mail.fetch(num, "(RFC822)")
    msg = email.message_from_bytes(data[0][1])
    subject = msg['subject']
    sender = msg['from']
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))
            try:
                body = part.get_payload(decode=True).decode()
            except:
                pass
    else:
        content_type = msg.get_content_type()
        body = msg.get_payload(decode=True).decode()
         # insert email data into MySQL database
    sql = "INSERT INTO emails (subject, sender, body) VALUES (%s, %s, %s)"
    val = (subject, sender, body)
    mycursor.execute(sql, val)
    mydb.commit()

mail.close()
mail.logout()

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])